# Exercise 2
Due:  Tue November 5, 8:00am

In [12]:
!pip install torch_geometric ogb torch-scatter

In [13]:
import torch
import torch_geometric as pyg
import numpy as np
import scipy
from ogb.graphproppred import PygGraphPropPredDataset
# from ogb.graphproppred.mol_encoder import AtomEncoder
import ogb
import torch_scatter
import copy

import time
import random
from tqdm import tqdm
import os

In this exercise, we use sparse message passing to make our networks scale to larger graphs. 


1) In this exercise we are working with the node-classification dataset Cora and the graph regression dataset ZINC. When working with a new dataset, it makes sense to at least quickly look into the data and some statistics for it. So for Cora: which is the second-biggest label class and what does it stand for? And for ZINC: how many HCO molecules (i.e. molecules consisting only of Hydrogen, Carbon, and Oxygen) are in the train set?

1) When working on the Cora dataset your model should at least reach an accuracy of 0.6 (an accuracy of 0.7-0.8 is well within reach).
Cora is a node classification dataset, so there is only one graph and we perform message passing on the whole graph (but evaluate the loss only on the nodes selected by cora_graph.train_mask).
The dataset is mostly balanced, so we evaluate the accuracy.
When implementing the message passing step, keep in mind that the graph does not contain self-loops (so one needs to somehow treat the "old" state).
Since Cora is small enough to be run with dense tensors too, you can verify your implementation this way.

2) ZINC is a small molecular regression dataset. Please compare the difference in performance between the (trainable) Atomencoder provided by ogb and the one-hot encoding you implemented in the first exercise.
Note that since you need batches, you need to modify the pooling layer to respect the batches.


In [14]:
# find device
if torch.cuda.is_available(): # NVIDIA
    device = torch.device('cuda')
elif torch.backends.mps.is_available(): # apple M1/M2
    device = torch.device('mps') 
else:
    device = torch.device('cpu')
device

device(type='cuda')

In [15]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed()


Random seed set as 42


## Cora

In [16]:
cora = pyg.datasets.Planetoid(root = "dataset/cora", name="Cora")
cora_graph = cora[0]
cora_dense_adj = pyg.utils.to_dense_adj(cora_graph.edge_index).to(device)
# cora_graph.x = cora_graph.x.unsqueeze(0) # Add an empty batch dimension. I needed that for compatibility with MolHIV later.
cora_graph = cora_graph.to(device)

Processing...
Done!


In [17]:
torch.unique(cora_graph.y)

tensor([0, 1, 2, 3, 4, 5, 6], device='cuda:0')

In [18]:
# dataloader with batch size 1
cora_loader = pyg.data.DataLoader([cora_graph], batch_size=1, shuffle=True)

/opt/conda/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [19]:
def get_accuracy(model, cora_loader, mask):
    model.eval()
    with torch.no_grad():
        accs = []
        for data in cora_loader:
            outputs = model(data.x, data.edge_index, data.batch)
            correct = (outputs[mask].argmax(-1) == data.y[mask]).sum()
            accs.append(int(correct) / int(mask.sum()))
        return np.mean(accs)

In [20]:
class GCNLayer(torch.nn.Module):
    def __init__(
        self,
        in_features: int,
        out_features: int,
        activation=torch.nn.functional.relu,
        reduction="mean",
    ):
        super(GCNLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.activation = activation
        self.reduction = reduction
        self.U = torch.nn.Sequential(
            torch.nn.Linear(in_features, out_features, bias=False),
            torch.nn.ReLU(),
            torch.nn.Linear(out_features, out_features, bias=False),
        )

        # Initialize weights
        torch.nn.init.kaiming_normal_(self.U[0].weight)
        torch.nn.init.kaiming_normal_(self.U[2].weight)

    def _add_remaining_self_loops(self, edge_index):
        num_nodes = edge_index.max().item() + 1
        mask = edge_index[0] != edge_index[1]
        loop_index = (
            torch.arange(num_nodes, device=edge_index.device).view(1, -1).repeat(2, 1)
        )
        edge_index = edge_index[:, mask]
        edge_index = torch.cat([edge_index, loop_index], dim=1)
        return edge_index

    def forward(self, H: torch.Tensor, edge_index: torch.Tensor):
        edge_index = self._add_remaining_self_loops(edge_index)
        Y = H[edge_index[0]]
        Z = pyg.utils.scatter(
            Y, edge_index[1], dim=0, dim_size=H.size(0), reduce=self.reduction
        )
        return self.activation(self.U(Z))

In [21]:
class GraphNet(torch.nn.Module):
    def __init__(
        self,
        in_features: int,
        out_features: int,
        hidden_features: int,
        num_layers=2,
        activation=torch.nn.functional.relu,
        dropout=0.1,
        reduction="mean",
        pooling="mean",
        task="node",
    ):
        super(GraphNet, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.hidden_features = hidden_features
        self.activation = activation
        self.dropout = dropout
        self.reduction = reduction
        self.pooling = pooling
        self.task = task

        self.layers = torch.nn.ModuleList(
            [
                GCNLayer(
                    in_features if i == 0 else hidden_features,
                    (
                        hidden_features
                        if (i < num_layers - 1) # if not the last layer
                        else (out_features if task == "node" else hidden_features) # if last layer: output features if node task, else hidden features as we will add a MLP
                    ),
                    activation,
                    reduction,
                )
                for i in range(num_layers)
            ]
        )
        
        if self.task != "node":
            self.bns = torch.nn.ModuleList(
                [
                    torch.nn.BatchNorm1d(
                        hidden_features
                        if i < num_layers - 1
                        else (out_features if task == "node" else hidden_features)
                    )
                    for i in range(num_layers)
                ]
            )

            self.mlp = torch.nn.Sequential(
                torch.nn.Linear(hidden_features, hidden_features),
                torch.nn.ReLU(),
                torch.nn.Dropout(dropout),
                torch.nn.Linear(hidden_features, out_features),
            )

    def forward(self, H: torch.Tensor, edge_index: torch.Tensor, batch: torch.Tensor):
        for i in range(len(self.layers)):
            H = self.layers[i](H, edge_index)
            if self.task != "node":
                H = self.bns[i](H)
                H = torch.nn.functional.dropout(H, p=self.dropout, training=self.training)
            
        
        if self.task != "node":
            # global pooling via scatter
            H = pyg.utils.scatter(
                H, batch, dim=0, dim_size=batch.max().item() + 1, reduce=self.pooling
            )
            H = self.mlp(H)
        return H

In [22]:
cora_num_classes = len(torch.unique(cora_graph.y))
model = GraphNet(
    in_features=cora_graph.num_node_features,
    out_features=cora_num_classes,
    hidden_features=32,
    num_layers=2,
    reduction="mean",
).to(device)

In [23]:
model

GraphNet(
  (layers): ModuleList(
    (0): GCNLayer(
      (U): Sequential(
        (0): Linear(in_features=1433, out_features=32, bias=False)
        (1): ReLU()
        (2): Linear(in_features=32, out_features=32, bias=False)
      )
    )
    (1): GCNLayer(
      (U): Sequential(
        (0): Linear(in_features=32, out_features=7, bias=False)
        (1): ReLU()
        (2): Linear(in_features=7, out_features=7, bias=False)
      )
    )
  )
)

In [24]:
# Training loop goes here
def train_cora(model, data_loader, epochs=200, lr=0.01, weight_decay=5e-4, device=device):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in tqdm(range(epochs)):
        val_accs = []
        train_accs = []
        for data in data_loader:
            model.train()
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, data.batch)
            print(out.shape)
            loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()   
            train_accs.append(get_accuracy(model, data_loader, data.train_mask))
            val_accs.append(get_accuracy(model, data_loader, data.val_mask))
        print(f"Epoch {epoch}, Validation Accuracy: {np.mean(val_accs)}, Training Accuracy: {np.mean(train_accs)}")

In [25]:
train_cora(model, cora_loader, epochs=200, lr=0.01, weight_decay=5e-4, device=device)

  0%|          | 0/200 [00:00<?, ?it/s]

torch.Size([2708, 7])


  4%|▍         | 8/200 [00:00<00:14, 13.22it/s]

Epoch 0, Validation Accuracy: 0.16, Training Accuracy: 0.2357142857142857
torch.Size([2708, 7])
Epoch 1, Validation Accuracy: 0.318, Training Accuracy: 0.45714285714285713
torch.Size([2708, 7])
Epoch 2, Validation Accuracy: 0.374, Training Accuracy: 0.5
torch.Size([2708, 7])
Epoch 3, Validation Accuracy: 0.37, Training Accuracy: 0.5857142857142857
torch.Size([2708, 7])
Epoch 4, Validation Accuracy: 0.416, Training Accuracy: 0.6357142857142857
torch.Size([2708, 7])
Epoch 5, Validation Accuracy: 0.47, Training Accuracy: 0.6642857142857143
torch.Size([2708, 7])
Epoch 6, Validation Accuracy: 0.5, Training Accuracy: 0.7357142857142858
torch.Size([2708, 7])
Epoch 7, Validation Accuracy: 0.592, Training Accuracy: 0.7857142857142857
torch.Size([2708, 7])
Epoch 8, Validation Accuracy: 0.636, Training Accuracy: 0.8214285714285714
torch.Size([2708, 7])
Epoch 9, Validation Accuracy: 0.654, Training Accuracy: 0.8285714285714286
torch.Size([2708, 7])
Epoch 10, Validation Accuracy: 0.676, Training Ac

 10%|█         | 21/200 [00:00<00:05, 32.51it/s]

...

Epoch 169, Validation Accuracy: 0.754, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 170, Validation Accuracy: 0.754, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 171, Validation Accuracy: 0.756, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 172, Validation Accuracy: 0.756, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 173, Validation Accuracy: 0.756, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 174, Validation Accuracy: 0.754, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 175, Validation Accuracy: 0.752, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 176, Validation Accuracy: 0.752, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 177, Validation Accuracy: 0.752, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 178, Validation Accuracy: 0.752, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 179, Validation Accuracy: 0.754, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 180, Validation Accuracy: 0.754, Training Accuracy: 1.0
torch.Size([27

 98%|█████████▊| 195/200 [00:03<00:00, 70.67it/s]

torch.Size([2708, 7])
Epoch 184, Validation Accuracy: 0.752, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 185, Validation Accuracy: 0.752, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 186, Validation Accuracy: 0.752, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 187, Validation Accuracy: 0.752, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 188, Validation Accuracy: 0.756, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 189, Validation Accuracy: 0.754, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 190, Validation Accuracy: 0.754, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 191, Validation Accuracy: 0.754, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 192, Validation Accuracy: 0.752, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 193, Validation Accuracy: 0.752, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 194, Validation Accuracy: 0.752, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 195, Validation Accuracy: 0.752, Training Accura

100%|██████████| 200/200 [00:03<00:00, 55.43it/s]

Epoch 198, Validation Accuracy: 0.754, Training Accuracy: 1.0
torch.Size([2708, 7])
Epoch 199, Validation Accuracy: 0.754, Training Accuracy: 1.0


In [26]:
get_accuracy(model, cora_loader, cora_graph.test_mask)

0.791

In [27]:
class GCNLayerDense(torch.nn.Module):
    def __init__(
        self,
        in_features: int,
        out_features: int,
        activation=torch.nn.functional.relu,
        skip_connection=False,
    ):
        super(GCNLayerDense, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.activation = activation
        self.skip_connection = skip_connection
        self.weight = torch.nn.Parameter(torch.FloatTensor(in_features, out_features))
        torch.nn.init.kaiming_normal_(self.weight)

    def forward(self, H: torch.Tensor, adj: torch.Tensor):
        h_x = torch.bmm(adj, torch.matmul(H, self.weight))
        if self.skip_connection:
            return self.activation(h_x + H)
        return self.activation(h_x)


class MeanPooling(torch.nn.Module):
    def __init__(self, dim: int | tuple[int, ...]):
        super(MeanPooling, self).__init__()
        self.dim = dim

    def forward(self, H: torch.Tensor):
        return H.mean(dim=self.dim)


class SumPooling(torch.nn.Module):
    def __init__(self, dim: int | tuple[int, ...]):
        super(SumPooling, self).__init__()
        self.dim = dim

    def forward(self, H: torch.Tensor):
        return H.sum(dim=self.dim)


class GraphGCNDense(torch.nn.Module):
    def __init__(
        self,
        num_layers: int,
        in_features: int,
        hidden_features: int,
        out_features: int,
        pooling: MeanPooling | SumPooling,
        activation=torch.nn.functional.relu,
        skip_connection: bool = False,
        # mlp_dropout_rate=0.1,
    ):
        super(GraphGCNDense, self).__init__()
        self.pooling = pooling
        self.activation = activation
        self.skip_connection = skip_connection
        self.layers = torch.nn.ModuleList(
            [
                GCNLayerDense(
                    in_features=in_features if i == 0 else hidden_features,
                    out_features=(
                        hidden_features if i < num_layers - 1 else out_features
                    ),
                    activation=activation,
                    skip_connection=skip_connection if (i != 0 and i != num_layers - 1) else False,
                )
                for i in range(num_layers)
            ]
        )
        # self.mlp = torch.nn.Sequential(
        #     torch.nn.Linear(hidden_features, hidden_features),
        #     torch.nn.ReLU(),
        #     torch.nn.Dropout(mlp_dropout_rate),
        #     torch.nn.Linear(hidden_features, out_features),
        # )

    def forward(self, H_in: torch.Tensor, adj: torch.Tensor):
        H = H_in
        for i in range(len(self.layers)):
            H = self.layers[i](H, adj)
        # H = self.pooling(H)
        # return self.mlp(H)
        return H


class GraphDataSetVectorized(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self._dataset = dataset
        self._largest_graph_size = int(dataset.get_summary().num_nodes.max)
        self.targets = self._dataset.data.y

    def __len__(self):
        return len(self._dataset)

    def __getitem__(self, idx):
        graph = self._dataset[idx]
        # print(graph)

        # adjacency matrix
        A = torch.zeros((self._largest_graph_size, self._largest_graph_size))
        # symmetric
        A[graph.edge_index[0], graph.edge_index[1]] = 1
        # self loop
        A = A + torch.eye(self._largest_graph_size)
        # Degree matrix
        D = torch.diag(torch.sum(A, axis=1))
        # Normalized adjacency matrix
        d_inv_sqrt = torch.pow(D, -0.5)
        d_inv_sqrt[torch.isinf(d_inv_sqrt)] = 0.0
        A_normalized = d_inv_sqrt @ A @ d_inv_sqrt

        # node features
        H = torch.zeros((self._largest_graph_size, graph.x.shape[1]))
        H[: graph.x.shape[0]] = graph.x

        # target
        target = graph.y
        return A_normalized, H, target, graph.train_mask, graph.val_mask, graph.test_mask

    def num_features(self):
        return self._dataset.num_features

    def compute_class_weights(self):
        class_counts = np.unique(self.targets, return_counts=True)[-1]
        frequencies = class_counts / len(self.targets)
        weights = np.round(1 / frequencies, 2)
        return torch.FloatTensor(weights / weights.sum()).to(device)

In [28]:
# Load the dataset
cora_dataset_vectorized = GraphDataSetVectorized(cora)
# print(len(cora_dataset_vectorized))
cora_loader_vectorized = torch.utils.data.DataLoader(
    cora_dataset_vectorized, batch_size=1, shuffle=True
)
# for A, H, target, train_mask, val_mask, test_mask in cora_loader_vectorized:
#     print(A.shape, H.shape, target.shape)
#     print(train_mask, val_mask, test_mask)
#     break

/opt/conda/lib/python3.10/site-packages/torch_geometric/data/summary.py:34: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/ReduceOps.cpp:1808.)
  std=data.std().item(),
/opt/conda/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [29]:
# Train the model using GraphGCNDense
model_dense = GraphGCNDense(
    num_layers=2,
    in_features=cora_dataset_vectorized.num_features(),
    hidden_features=32,
    out_features=cora_num_classes,
    pooling=MeanPooling(dim=1),
    activation=torch.nn.functional.relu,
    skip_connection=True,
).to(device)
# print(model_dense)
optimizer_dense = torch.optim.Adam(model_dense.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn_dense = torch.nn.CrossEntropyLoss()
num_epochs = 200

for epoch in tqdm(range(num_epochs)):
    val_accs = []
    train_accs = []
    losses = []
    for adj, features, target, train_mask, val_mask, test_mask in cora_loader_vectorized:
        model_dense.train()
        adj, features, target = adj.to(device), features.to(device), target.to(device)
        train_mask, val_mask, test_mask = train_mask.to(device), val_mask.to(device), test_mask.to(device)
        model_dense.train()
        optimizer_dense.zero_grad()
        out = model_dense(features, adj)
         
        loss = loss_fn_dense(out[train_mask], target[train_mask])
        loss.backward()
        optimizer_dense.step()
        train_acc = out[train_mask].argmax(-1) == target[train_mask]
        train_accs.append(train_acc.float().sum() / train_mask.sum())
        val_acc = out[val_mask].argmax(-1) == target[val_mask]
        val_accs.append(val_acc.float().sum() / val_mask.sum())
        losses.append(loss.item())
    print(f"Epoch {epoch}, Validation Accuracy: {torch.mean(torch.tensor(val_accs))}, Training Accuracy: {torch.mean(torch.tensor(train_accs))}, Loss: {torch.mean(torch.tensor(losses))}")


  0%|          | 1/200 [00:00<01:32,  2.16it/s]

Epoch 0, Validation Accuracy: 0.20399999618530273, Training Accuracy: 0.2142857164144516, Loss: 2.1051907539367676


  1%|          | 2/200 [00:00<01:21,  2.43it/s]

Epoch 1, Validation Accuracy: 0.25999999046325684, Training Accuracy: 0.4000000059604645, Loss: 1.7094868421554565


  2%|▏         | 3/200 [00:01<01:18,  2.51it/s]

...

Epoch 197, Validation Accuracy: 0.7680000066757202, Training Accuracy: 1.0, Loss: 0.008197390474379063


100%|█████████▉| 199/200 [01:18<00:00,  2.56it/s]

Epoch 198, Validation Accuracy: 0.765999972820282, Training Accuracy: 1.0, Loss: 0.008186987601220608


100%|██████████| 200/200 [01:19<00:00,  2.53it/s]

Epoch 199, Validation Accuracy: 0.765999972820282, Training Accuracy: 1.0, Loss: 0.008177277632057667


In [30]:
# Evaluate the model
model_dense.eval()
with torch.no_grad():
    accs = []
    for adj, features, target, train_mask, val_mask, test_mask in cora_loader_vectorized:
        adj, features, target = adj.to(device), features.to(device), target.to(device)
        out = model_dense(features, adj)
        correct = (out[test_mask].argmax(-1) == target[test_mask]).sum()
        accs.append(int(correct) / int(test_mask.sum()))
    print(f"Test Accuracy: {np.mean(accs)}")

Test Accuracy: 0.797


In [31]:
# from torch_geometric.nn import GCNConv
# import torch.nn.functional as F

# class GCN(torch.nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = GCNConv(1433, 16) # shape（输入的节点特征维度 * 中间隐藏层的维度）
#         self.conv2 = GCNConv(16, 7) # shape（中间隐藏层的维度 * 节点类别）

#     def forward(self, data):
#         x, edge_index = data.x, data.edge_index # 加载节点特征和邻接关系

#         x = self.conv1(x, edge_index) # 传入卷积层
#         x = F.relu(x) # 激活函数
#         x = F.dropout(x, training=self.training) # dropout层，防止过拟合
#         x = self.conv2(x, edge_index) # 第二层卷积层

#         # 将经过两层卷积得到的特征输入log_softmax函数得到概率分布
#         return F.log_softmax(x, dim=1)

In [32]:
# m2 = GCN().to(device)
# m2

In [33]:
# optimizer = torch.optim.Adam(m2.parameters(), lr=0.01, weight_decay=5e-4)
# num_epoch = 200

# m2.train()
# for epoch in range(50):
#     optimizer.zero_grad() # 梯度清空为0
#     out = m2(cora_graph) # 模型输出
#     print(out.shape)
#     loss = F.nll_loss(out[cora_graph.train_mask], cora_graph.y[cora_graph.train_mask]) # 计算损失函数
#     correct = (out.argmax(dim=1)[cora_graph.train_mask] == cora_graph.y[cora_graph.train_mask]).sum() # 计算正确的个数
#     acc = int(correct) / int(cora_graph.train_mask.sum()) # 得出准确率
#     loss.backward() # 反向传播计算梯度
#     optimizer.step() # 利用梯度更新模型参数
#     if (epoch+1) % 10 == 0:
#         print('Epoch: {}, Loss: {:.4f}, Training Acc: {:.4f}'.format(epoch+1, loss.item(), acc))



## ZINC

In [ ]:
# Load the dataset
dataset = pyg.datasets.ZINC(root='dataset/ZINC', split='train', subset=True)
dataset_val = pyg.datasets.ZINC(root='dataset/ZINC', split='val', subset=True)
dataset_test = pyg.datasets.ZINC(root='dataset/ZINC', split='test', subset=True)
print(dataset[0])
# Create data loaders
batch_size=128
num_workers = 8
train_loader = pyg.loader.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = pyg.loader.DataLoader(dataset_val, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = pyg.loader.DataLoader(dataset_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

Extracting dataset/ZINC/molecules.zip
Processing...
/opt/conda/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.

In [ ]:
for data in train_loader:
    print(data)
    break

In [ ]:
# your implementation goes here